# 函数式编程

#### 一种抽象计算的编程模式

Python的函数式编程：
- 高阶函数
- 闭包
- 匿名函数
- 装饰器

## 高阶函数

变量可以指向函数

In [ ]:
abs(-10)

In [ ]:
fun_abs = abs

函数本身也可以赋值给变量，即：变量可以指向函数。

In [ ]:
fun_abs(-10)

一个函数就可以接收另一个函数作为参数，这种函数就称之为高阶函数。

In [ ]:
def add(x, y, f):
    return f(x) + f(y)

In [ ]:
add(-5, -6, abs)

### map

map()是 Python 内置的高阶函数，它接收一个函数 f 和一个 list，并通过把函数 f 依次作用在 list 的每个元素上

In [ ]:
def f(x):
    return x * x
print(map(f, [1, 2, 3, 4, 5, 6]))

### reduce

reduce()把一个函数作用在一个序列[x1, x2, x3, ...]上，这个函数必须接收两个参数，reduce把结果继续和序列的下一个元素做累积计算

In [ ]:
from functools import reduce
def add(x, y):
    return x + y
reduce(add, [1, 2, 3, 4, 5])

## filter

filter()函数接收一个函数 f 和一个list，这个函数 f 的作用是对每个元素进行判断，返回 True或 False

In [ ]:
def is_odd(n):
    return n % 2 == 1
list(filter(is_odd, [1,2,3,4,5,6,10,15]))

## sorted

In [ ]:
sorted([36, 5, -12, 9, -21])

实现自定义排序

In [ ]:
# cmp_to_key
# 将老式的比较函数（comparison function）转换为关键字函数（key function）
# 该函数主要用于将程序转换成Python 3格式的，因为Python 3中不支持比较函数
from functools import cmp_to_key
def reversed_cmp(x, y):
    if x > y:
        return -1
    if x < y:
        return 1
    return 0
sorted([36, 5, -12, 9, -21], key=cmp_to_key(reversed_cmp))

## 返回函数

高阶函数除了可以接受函数作为参数外，还可以把函数作为结果值返回。

In [ ]:
def myabs():
    return abs   # 返回函数
def myabs2(x):
    return abs(x)   # 返回函数调用的结果，返回值是一个数值

返回函数可以把一些计算延迟执行

In [ ]:
def calc_sum(lst):
    def lazy_sum():
        return sum(lst)
    return lazy_sum

In [ ]:
f = calc_sum([1, 2, 3, 4])
f

In [ ]:
f()

### 闭包

内层函数引用了外层函数的变量（参数也算变量），然后返回内层函数的情况，称为闭包（Closure）

闭包的特点是返回的函数还引用了外层函数的局部变量，所以，要正确使用闭包，就要确保引用的局部变量在函数返回后不能变。

In [ ]:
# 希望一次返回3个函数，分别计算1x1,2x2,3x3:
def count():
    fs = []
    for i in range(1, 4):
        def f():
             return i*i
        fs.append(f)
    return fs

f1, f2, f3 = count()

In [ ]:
f2()

因此，返回函数不要引用任何循环变量，或者后续会发生变化的变量

## 匿名函数 lambda

In [ ]:
list(map(lambda x: x * x, [1, 2, 3, 4, 5, 6, 7, 8, 9]))

lambda x: x * x实际上就是

In [ ]:
def f(x):
    return x * x

Python对匿名函数的支持有限，只有一些简单的情况下可以使用匿名函数。

## 装饰器

接收一个函数，对其包装，然后返回一个新函数

In [53]:
def f1(x):
    return x*2
def new_fn(f):     #装饰器函数
    def fn(x):
        print("call "+f.__name__+"()")
        return f(x)
    return fn

In [54]:
# 调用
g1 = new_fn(f1)
print(g1(5))

call f1()
10


In [ ]:
# 隐藏原函数
f1 = new_fn(f1)
print(f1(5))

装饰器语法@，简化调用

In [ ]:
@new_fn
def f1(x):
    return x*2
f1(5)

作用：
- 打印日志：@log
- 检测性能：@performance
- 数据库事务：@transaction
- URL路由：@port('/api')

### 带参数decorator

In [ ]:
@log('DEBUG')
def my_func():
    pass

等同于

In [ ]:
my_func = log('DEBUG')(my_func)

等同于

In [ ]:
log_decorator = log('DEBUG')
my_func = log_decorator(my_func)

等同于

In [ ]:
log_decorator = log('DEBUG')
@log_decorator
def my_func():
    pass

所以，带参数的log函数首先返回一个decorator函数，再让这个decorator函数接收my_func并返回新函数：

In [57]:
def log(prefix):
    def log_decorator(f):
        def wrapper(*args, **kw):
            print('[%s] %s()...' % (prefix, f.__name__))
            return f(*args, **kw)
        return wrapper
    return log_decorator

In [59]:
@log('DEBUG')
def test():
    pass
print(test())

[DEBUG] test()...
None


#### 完善decorator

加上装饰器后原函数的属性会改变

In [61]:
print(test.__name__)

wrapper


借助functools完成函数属性复制

In [66]:
import functools
def log(prefix):
    def log_decorator(f):
        @functools.wraps(f)
        def wrapper(*args, **kw):
            print('[%s] %s()...' % (prefix, f.__name__))
            return f(*args, **kw)
        return wrapper
    return log_decorator

In [67]:
@log('DEBUG')
def test():
    pass
print(test.__name__)

test


## 偏函数

当一个函数有很多参数时，调用者就需要提供多个参数。如果减少参数个数，就可以简化调用者的负担

int()函数可以把字符串转换为整数，当仅传入字符串时，int()函数默认按十进制转换：

In [ ]:
int('12345')

In [ ]:
int('12345', base=8)

In [ ]:
int('12345', 16)

假设要转换大量的二进制字符串，每次都传入int(x, base=2)非常麻烦，可以定义一个int2()的函数，默认把base=2传进去：

In [ ]:
def int2(x, base=2):
    return int(x, base)

In [ ]:
int2('1000000')

用functools.partial创建一个偏函数的

In [ ]:
import functools
int2 = functools.partial(int, base=2)

In [ ]:
int2('1000000')

简单总结functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数。